In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install python-binance -qq

In [ ]:
import pandas as pd
import numpy 
from tqdm.notebook import tqdm
import pickle
import pandas as pd
import math
import os.path
import time
import requests
#from bitmex import bitmex
from binance.client import Client
#from binance.spot import Spot as Client
import os

from datetime import timedelta, datetime
from dateutil import parser
from tqdm import tqdm_notebook #(Optional, used for progress-bars)

from IPython.display import clear_output

In [ ]:
#os.environ ["http_proxy"]='http://194.5.193.183:80'

### API
#bitmex_api_key = 'UGaVHVUZBILUINmiYk3bs4d6'    #Enter your own API-key here
#bitmex_api_secret = 'nMetNhylbIaUeqdgkl9bdEHnSuxR1Ov0bzI6YuWhUJ2AqC5b' #Enter your own API-secret here
binance_api_key = binance_api    #Enter your own API-key here
binance_api_secret = binance_secret #Enter your own API-secret here

#!export HTTP_PROXY="http://10.10.1.10:3128"
#!export HTTPS_PROXY="http://10.10.1.10:1080"

while True:
    try:
        binance_client = Client(api_key=binance_api_key, api_secret=binance_api_secret)
        break
    except Exception as e:
        print(e)

In [ ]:
### CONSTANTS
binsizes = {"1m": 1, "3m": 3, "5m": 5, "10m": 10, "15m": 15, "30m": 30,
            "1h": 60, "3h": 180, "6h": 360, "12h": 720, "1d": 1440}
batch_size = 750

### FUNCTIONS
def minutes_of_new_data(symbol, kline_size, data, source):
    if len(data) > 0:  old = parser.parse(data["timestamp"].iloc[-1])
    elif source == "binance": old = datetime.strptime('1 Jan 2019', '%d %b %Y')
    elif source == "bitmex": old = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=False).result()[0][0]['timestamp']
    if source == "binance": new = pd.to_datetime(binance_client.get_klines(symbol=symbol, interval=kline_size)[-1][0], unit='ms')
    if source == "bitmex": new = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=True).result()[0][0]['timestamp']
    return old, new

def get_all_binance(symbol, kline_size, save = False):
    filename = '/content/drive/MyDrive/generate_dataset/movement/dataset/%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): 
      data_df = pd.read_csv(filename)
      data_df = data_df[:-500]
    else: data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "binance")
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    if oldest_point == datetime.strptime('1 Jan 2021', '%d %b %Y'): print('Downloading all available %s data for %s. Be patient..!' % (kline_size, symbol))
    else: print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data.' % (delta_min, symbol, available_data, kline_size))
    klines = binance_client.get_historical_klines(symbol, kline_size, oldest_point.strftime("%d %b %Y %H:%M:%S"), newest_point.strftime("%d %b %Y %H:%M:%S"))
    data = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    if len(data_df) > 0:
        temp_df = pd.DataFrame(data)
        data_df = data_df.append(temp_df)
        data_df = data_df[:-1]
    else: data_df = data[:-1]
    data_df.set_index('timestamp', inplace=True)
    if save: data_df.to_csv(filename)
    print('All caught up..!')
    return data_df


In [ ]:
%%time
get_all_binance("ETHBUSD", '1d', save = True)
get_all_binance("ETHBUSD", '12h', save = True)
get_all_binance("ETHBUSD", '6h', save = True)
get_all_binance("ETHBUSD", '1h', save = True)
get_all_binance("ETHBUSD", '30m', save = True)
get_all_binance("ETHBUSD", '15m', save = True)
get_all_binance("ETHBUSD", '5m', save = True)

In [ ]:
while (True):
  get_all_binance("ETHBUSD", '1d', save = True)
  get_all_binance("ETHBUSD", '12h', save = True)
  get_all_binance("ETHBUSD", '6h', save = True)
  get_all_binance("ETHBUSD", '1h', save = True)
  get_all_binance("ETHBUSD", '30m', save = True)
  get_all_binance("ETHBUSD", '15m', save = True)
  get_all_binance("ETHBUSD", '5m', save = True)
  clear_output()